# **[Tableau dashboard for the current clients NPS level](https://public.tableau.com/app/profile/irina.ganina5744/viz/NPS_Dashboard_16825385674160/NPSDashboard)**

In [9]:
# importing libraries 

import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine, text 

In [13]:
# configuring the database connection

path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = 'https:/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    conn = engine.connect()

In [14]:
# sql query to form the dataframe

query = text("""
WITH
is_new AS (SELECT user_id,
            CASE
                WHEN lt_day < 366 THEN 'yes'
                ELSE 'no'
            END AS is_new
            FROM user
            WHERE lt_day IS NOT NULL),
nps AS (SELECT user_id,
            CASE
                WHEN nps_score > 8 THEN 'promoter'
                WHEN nps_score == 7 or nps_score == 8 THEN 'passive'
                ELSE 'detractor'
            END AS nps_group
            FROM user
            WHERE nps_score IS NOT NULL),
gender AS (SELECT user_id,
            CASE
                WHEN gender_segment == 1 THEN 'female'
                ELSE 'male'
            END AS gender_segment
            FROM user
            WHERE gender_segment IS NOT NULL)
                
SELECT user.user_id,
    user.lt_day,
    is_new.is_new,
    user.age,
    gender.gender_segment,
    user.os_name,
    user.cpe_type_name,
    location.country,
    location.city,
    age_segment.title AS age_segment,
    traffic_segment.title AS traffic_segment,
    lifetime_segment.title AS lifetime_segment,
    user.nps_score,
    nps.nps_group
    
FROM user LEFT JOIN location ON user.location_id = location.location_id
    LEFT JOIN age_segment ON user.age_gr_id = age_segment.age_gr_id
    LEFT JOIN traffic_segment ON user.tr_gr_id = traffic_segment.tr_gr_id
    LEFT JOIN lifetime_segment ON user.lt_gr_id = lifetime_segment.lt_gr_id
    LEFT JOIN is_new ON user.user_id = is_new.user_id
    LEFT JOIN nps ON user.user_id = nps.user_id
    LEFT JOIN gender ON user.user_id = gender.user_id
""")

In [16]:
# creating a pandas dataframe

df = pd.read_sql(query, conn)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,no,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoter
1,A001WF,2344,no,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoter
2,A003Q7,467,no,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoter
3,A004TB,4190,no,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoter
4,A004XT,1163,no,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoter


In [17]:
# checking df info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [18]:
# checking for duplicates

df.duplicated().sum()

0

In [19]:
# checking for missing values

df.isna().sum()

user_id                0
lt_day                 0
is_new                 0
age                  554
gender_segment      1301
os_name                0
cpe_type_name          0
country                0
city                   0
age_segment            0
traffic_segment        0
lifetime_segment       0
nps_score              0
nps_group              0
dtype: int64

Since the project description does not provide for data preprocessing, I will move on to creating a dashboard in Tableau based on the obtained table. Note that there are gaps in the age and gender_segment columns, but we will not fill them in.